In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import math
import numpy as np

In [ ]:
NEWS_FILE = 'data/news_reuters.csv'

In [ ]:
col_names = ['code','name','date','headline','article','importance']
news = pd.read_csv(NEWS_FILE,names=col_names)

In [ ]:
news 

In [ ]:
# 清洗数据(词性还原)
def clean_news(news_raws):
    lemma=WordNetLemmatizer()
    news_rows=[]
    for i in range(0,len(news_raws)):
        news=str(news_raws[i])
        news=re.sub('[^a-zA-Z]', ' ',news) 
        news=[lemma.lemmatize(w) for w in word_tokenize(str(news).lower())]  # 词性还原
        news=' '.join(news)
        news_rows.append(news)
    return news_rows

In [ ]:
news['text'] = clean_news(news['headline']+news['article'])

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(news['text'] )

In [ ]:
word=vectorizer.get_feature_names()

In [ ]:
def remove_deplicates(news_file,rad=10,threshold = 0.9):
    dp_list = set()
    tops = news.loc[news['importance']=='topStory']
    for index in tops.index:
        top = list()
        top.append(str(news['text'].loc[index]))
        li = list(range(index-rad,index+rad+1))
        win = [ str(item) for item in news['text'].loc[li]]
        top = vectorizer.transform(top).toarray()[0]
        win = vectorizer.transform(win).toarray()
        for i in range(len(win)):
            cur = win[i]
            up = np.dot(top,cur)
            down = math.sqrt(np.dot(top,top))*math.sqrt(np.dot(cur ,cur))
            if down == 0 :
                continue
            sim = up/down
            if sim > threshold and rad!=i:
                dp_list.add(index-rad+i)
    return dp_list

In [ ]:
dp_list = remove_deplicates(news)

In [ ]:
news2 = news.drop(list(dp_list))

In [ ]:
news2 = news2.drop(['name','date','headline','article'],axis=1)